In [3]:
import json
import random
import cv2
import matplotlib.pyplot as plt
import shutil
import os

In [2]:
target_cnt = 1000     # target_cnt 미만인 class들을 target_cnt 근처로 개수를 맞춰줌
maximum_IoU = 0.3     # 합성할 때 배경 이미지의 객체들과 겹치는 허용 정도

In [3]:
random.seed(10)

In [4]:
def get_file_path(json_file, image_num):
    file_path = json_file['images'][image_num]['file_name']
    return file_path


def get_annotations(json_file, image_num):
    anns = [ann['bbox'] for ann in json_file['annotations'] if ann['image_id'] == image_num]
    return anns


def read_image(dataset_path, file_path):
    image = cv2.imread(dataset_path + file_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


def plot_bbox(image, anns):
    for ann in anns:
        xmin, ymin, w, h = map(int, ann['bbox'])
        print(xmin, ymin, w, h)
        image = cv2.rectangle(image, (xmin, ymin), (xmin + w, ymin + h), (0, 0, 255), 3)
    plt.imshow(image)

def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

In [5]:
os.makedirs("../dataset/new_train", exist_ok=True)

In [6]:
# train 이미지들 new_train 폴더에 복사
for img_dir in os.listdir('../dataset/train'):
    shutil.copyfile(f"../dataset/train/{img_dir}", f"../dataset/new_train/{img_dir}")

In [7]:
with open('../dataset/train.json', 'r') as f:
    train_json = json.load(f)
dataset_path = '../dataset/'

In [8]:
# train_josn의 이미지 경로 /train -> /new_train으로 바꿈
for x, i in enumerate(train_json['images']):
    train_json['images'][x]['file_name'] = train_json['images'][x]['file_name'].replace('train', 'new_train')

In [9]:
# 1000개 미만인 class들만 list로 각 클래스 별로 annotation 정보 담기
section = [[i for i in train_json['annotations'] if i['category_id'] == j] for j in range(10)]
low_quantity_class = [x for x, i in enumerate(section) if len(i) < target_cnt]
low_quantity_annotations = [[i for i in train_json['annotations'] if i['category_id'] == j] for j in low_quantity_class]
[len(i) for i in section]

[3966, 6352, 897, 936, 982, 2943, 1263, 5178, 159, 468]

In [10]:
for x, low_classes in enumerate(low_quantity_annotations):
    for i in range(1000 // len(low_classes)):
        for low_class in low_classes:
            # 배경이미지가 5개 미만인 것이 뽑힐 때까지 랜덤으로 뽑기
            while True:
                background_img_num = random.randint(0, len(train_json['images'])-1)
                background_anns = get_annotations(train_json, background_img_num)
                if len(background_anns) < 5: break

            background_img_path = get_file_path(train_json, background_img_num)
            background_img = read_image(dataset_path, background_img_path)
            
            object_img_path = get_file_path(train_json, low_class['image_id'])
            object_img = read_image(dataset_path, object_img_path)
            object_img_bbox = map(int, low_class['bbox'])

            xmin, ymin, w, h = object_img_bbox
            object_img_cut = object_img[ymin:ymin+h, xmin:xmin+w].copy()

            # 합성할 이미지와 배경이미지들의 객체의 IoU가 다 0.2 미만일 때까지 랜덤 위치 추출
            cnt = 0
            while True:
                new_xmin, new_ymin = random.randint(0, 1025 - w - 1), random.randint(0, 1025 - h - 1)
                new_box = [new_xmin, new_ymin, new_xmin+w, new_ymin+h]
                ann = background_anns[0]
                check = [True if IoU(new_box, [ann[0], ann[1], ann[0]+ann[2], ann[1]+ann[3]]) < maximum_IoU else False 
                        for ann in background_anns]
                cnt += 1
                if all(check) or cnt == 1000: break
            if cnt == 1000: continue

            # 추가한 부분 train_json의 annotations부분에 append
            train_json['annotations'].append({
                                "image_id": background_img_num,
                                "category_id": low_quantity_class[x],
                                "area": low_class['area'],
                                "bbox": [new_xmin, new_ymin, w, h],
                                "iscrowd": 0,
                                "id": len(train_json['annotations'])
                                })
            combine_img = background_img.copy()
            combine_img[new_ymin:new_ymin+h, new_xmin:new_xmin+w] = object_img_cut
            # print(background_img_path) # 합성한 이미지 경로 보기
            cv2.imwrite(os.path.join('../dataset/', background_img_path), combine_img)          

In [11]:
# 수정된 train_json을 new_train.json파일로 저장
with open('../dataset/new_train.json', 'w') as f:
    json.dump(train_json, f, indent="\t")

In [12]:
# 각 클래스별 개수 보기
section = [[i for i in train_json['annotations'] if i['category_id'] == j] for j in range(10)]
[len(i) for i in section]

[3966, 6352, 1792, 1869, 1959, 2943, 1263, 5178, 1113, 1368]